In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# computational dynamic graph is a ds that helps tracking changes in a tensor using graphs ds. The next code sets up computational tracking on a tensor
x = torch.tensor(2.0, requires_grad=True)

In [3]:
# defining a tensorial function 
y = 2*x**4 + x**3 + 3*x**2 + 5*x + 1
print(y)

tensor(63., grad_fn=<AddBackward0>)


In [8]:
def f(x):
    return 3*x**2 - 4*x

def numerical_lim(f, x, h):
    return (f(x+h)-f(x))/h

h = 0.1
for i in range(7):
    print(f'h={h: 6f}, numerical limit = {numerical_lim(f, 1, h):.6f}')
    h *= 0.1

h= 0.100000, numerical limit = 2.300000
h= 0.010000, numerical limit = 2.030000
h= 0.001000, numerical limit = 2.003000
h= 0.000100, numerical limit = 2.000300
h= 0.000010, numerical limit = 2.000030
h= 0.000001, numerical limit = 2.000003
h= 0.000000, numerical limit = 2.000000
